In [25]:
import time
import math
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt 
import csv
import random
import json
from pathlib import Path

In [26]:
def qubo_formulation(G, a, d):

    start_time = time.process_time()
    n = G.order()
    m = G.size()
    
    #z_binary = maximum number of binary bits needed to represent any edge label, z = biggest possible edge label
    z = a + (n-2)*d
    z_binary = math.ceil(math.log2(z))

    incidence_matrix = nx.incidence_matrix(G).toarray()

    Q = {}
    offset = n
    #for each vertex:
    for i in range(n):
        #degree
        degree = sum(incidence_matrix[i])
        offset += (degree - a)**2
        #F1^2
        for j in range(m):       
            mij = incidence_matrix[i][j]
            if mij == 1:   
                for k in range(z_binary):
                    #linear part
                    if Q.get((f'y{j}{k}',f'y{j}{k}')) == None:
                        Q[(f'y{j}{k}',f'y{j}{k}')]= (2**k)**2-2*a*(2**k)+2*degree*(2**k)
                    else:
                        Q[(f'y{j}{k}',f'y{j}{k}')]+= (2**k)**2-2*a*(2**k)+2*degree*(2**k)

                    #quadratic part
                    for jj in range(j,m):
                        mijj=incidence_matrix[i,jj]
                        if mijj == 1:
                            for kk in range(z_binary):     
                                if k<kk <= z_binary or j<jj<=m:
                                    if Q.get((f'y{j}{k}',f'y{jj}{kk}')) == None:
                                        Q[(f'y{j}{k}',f'y{jj}{kk}')]= 2*(2**k)*(2**kk)
                                    else:
                                        Q[(f'y{j}{k}',f'y{jj}{kk}')]+= 2*(2**k)*(2**kk)
        for h in range(n):
            #F2^2
            if Q.get((f'T{i}{h}',f'T{i}{h}')) == None:
                Q[(f'T{i}{h}',f'T{i}{h}')] = d**2*h**2 + 2*a*d*h - 2*h*d*degree
            for h2 in range(h+1,n):
                Q[(f'T{i}{h}',f'T{i}{h2}')] = 2*d**2*h*h2

            #-2F1F2
            for j in range(m):
                mij = incidence_matrix[i][j]
                if mij == 1:
                    for k in range(z_binary):
                        Q[(f'y{j}{k}',f'T{i}{h}')] = -2*h*2**k*d

    #Constraints P
    for i in range(n):
        for h in range(n):
            for h2 in range(h+1,n):
                if Q.get((f'T{i}{h}',f'T{i}{h2}')) == None:
                    Q[(f'T{i}{h}',f'T{i}{h2}')] = 1
                else:
                    Q[(f'T{i}{h}',f'T{i}{h2}')] += 1
    for i in range(n):
        for h in range(n):
            for i2 in range(i+1,n):
                if Q.get((f'T{i}{h}',f'T{i2}{h}')) == None:
                    Q[(f'T{i}{h}',f'T{i2}{h}')] = 1
                else:
                    Q[(f'T{i}{h}',f'T{i2}{h}')] += 1           
    for i in range(n):
        for h in range(n):
            if Q.get((f'T{i}{h}',f'T{i}{h}')) == None:
                Q[(f'T{i}{h}',f'T{i}{h}')] = -1
            else:
                Q[(f'T{i}{h}',f'T{i}{h}')] += -1
    elapsed_time = (time.process_time() - start_time)*(10**6)
    return Q, offset, elapsed_time

In [27]:
#Read the DataFrame from input_file_path, append columns for qubos, offsets, and runtime, and then save the updated DataFrame to output_file_path.
#write_qubos (input_file_path, output_file_path)
def write_qubos(input_file, output_file,):
    
    df = pd.read_csv(input_file)
    new_df = df.iloc[:, :-1].copy()
    
    qubos = []
    offsets = []
    runtimes = []
    
    n=df.shape[0]
    for i in range(n):
        a = df['a'][i]
        d = df['d'][i]

        # Build a NetworkX graph using the provided adjacency list.        
        ad_str = df['Adjacency_list'][i]
        ad_dict=json.loads(ad_str)
        ad_list = pd.DataFrame(ad_dict)
        G = nx.from_pandas_adjacency(ad_list)
        
        #take the average time among n times of formulations
        n = 10
        runtime_total = 0
        for i in range(n):
            
            #get qubo, offset_value, runtime for qubo formulation from function qubo_formulation
            Q, offset, formulation_time = qubo_formulation(G,a,d)
            
            runtime_total += formulation_time
            
        qubos.append(Q)
        offsets.append(offset)
        runtimes.append(runtime_total/n)
        
    #write qubo, offset, runtime to new_df
    new_df['qubo'] = qubos
    new_df['offset'] = offsets
    new_df['qubo_formulation_time/microseconds'] = runtimes
    new_df.to_csv(output_file, index=False)


In [32]:
write_qubos('apgs/apg8.csv','apglp_qubos/apglp8_qubo.csv')